# Fitting a distribution to waiting times

### How long should we wait for the bus before giving up on it and starting to walk?

First, we'll need to observe some data on the historic arrival times of the bus and fit a distribution to them. Note however that some of our data will be incomplete since when we give up on the bus after x minutes, we only know it took more than that time for it to arrive, but not exactly how much. These are called censored observations.

Let's generate sample data - both complete observations (ti) and some censored observations (xi) - and fit a distribution! 


In [ ]:
import matplotlib.pyplot as plt
from distributions.lomax import *
from distributions.loglogistic import *

In [ ]:
# Define parameters for Lomax

k = 10.0   # shape
lmb = 0.5  # scale
sample_size = 5000
censor_level = 0.5 # after half an hour, we stop waiting. 
prob = 1.0

In [ ]:
# Let's assume the arrival times of the bus follow a Lomax distribution.
l = Lomax(k=k, lmb=lmb)

### What is lomax distribution?

It is basically a Pareto distribution that has been shifted so that its support begins at zero. A heavy tailed distribution. For a non-negative random variable.

Two parameters define the distribution: scale parameter λ and shape parameter κ (sometimes denoted as α). 

The shorthand X ∼ Lomax(λ,κ) indicates the random variable X has a Lomax distribution with those two parameters.


<img src = "https://www.statisticshowto.datasciencecentral.com/wp-content/uploads/2016/06/lomax-pdf.png">

<img src = "https://www.statisticshowto.datasciencecentral.com/wp-content/uploads/2016/06/PDF.png">

In [ ]:
# Generate waiting times from Lomax distribution.
samples = l.samples(size=sample_size)
samples

In [ ]:
# Since we never wait for the bus more than x minutes,
# the observed samples are the ones that take less than x minutes.
ti = samples[(samples<=censor_level)]
ti

In [ ]:
len(ti) # About 10% of people stopped waiting after 30 minutes.

In [ ]:
samples > censor_level


In [ ]:
# xi array contains the censored data.
xi = np.ones(sum(samples>censor_level)) * censor_level
xi

In [ ]:
len(xi) 

In [ ]:
# Fit a log logistic model to the data we just generated.
# You can safely ignore the warnings.

ll1 = LogLogistic(ti=ti, xi=xi)

In [ ]:
# See how well the distribution fits the histogram.

histo = plt.hist(samples, normed=True)
xs = (histo[1][:len(histo[1])-1]+histo[1][1:])/2

In [ ]:
xs # We are going to call pdf for each xs values.

In [ ]:
plt.plot(xs, [ll1.pdf(i) for i in xs])
plt.show()

### Optimizing waiting threshold using the distribution

Let's model the process as a state machine. There are three possible states that we care about - "1. waiting for a bus", "2. walking to work" and "3. working at the office". The figure below represents the states and the arrows show the possible transitions between the states.


<img src="https://raw.githubusercontent.com/ryu577/ryu577.github.io/master/Downloads/opt_thresholds/bus_states.png" width="480" height="400" >


Also, we assume that which state we go to next and how much time it takes to jump to that state depends only on *which state we are currently in*. This property is called the **Markov property**. 


To describe this transitions, we need two matrices.

1. Transition probabilities : the transition from state 'i' to state 'j' 
\n
2. Transition times

the first state (i = 0) is "waiting", the second state (i=1) is "walking" and the last and most desirable state (i = 2) is "working", where we want to spend the highest proportion of time.

Continuing from above, we can run the following code:


In [ ]:
# The time it takes to walk to work 
intervention_cost=200

# The amount of time we wait for the bus before walking respectively.
tau=275

# The transition probabilities (p) and transition times (t) depend on 
# the amount of time we're willing to wait for the bus (tau)
# and the amount of time it takes to walk to work (intervention_cost).
(p,t) = ll1.construct_matrices(tau, intervention_cost)

In [59]:
# The transition probabilities
p

matrix([[0.00000000e+00, 3.91744301e-05, 9.99960826e-01],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.00000000e-01, 9.00000000e-01, 0.00000000e+00]])

The 'p' matrix you see above is the matrix of transition probabilities. The (i,j) row is the probability of transitioning to state 'j' given you started in state 'i'. Note that the rows of this matrix sum to 1 since we have to make a transition to one of the other available states. Also, the diagonals of the matrix are 0 since the (i,i) entry would imply transitioning from state i to i, which doesn't make sense in the context of a transition. 


In [58]:
#transition times
t

matrix([[  0.        , 275.        ,   0.40033456],
        [  0.        ,   0.        , 200.        ],
        [100.        , 100.        ,   0.        ]])

Given this transition matrix, let's say we start in any state i and make a transition to another state according to the probabilities given by row 'i' in the matrix. If we end up in state 'j', we spend one unit of time there and then make another random transition according to the probabilities in row 'j' and so on, repeating this process many times. What percentage of the total time would we then expect to spend in each of the states? This is called the vector of steady state probabilities and it can be calculated via the method described in the answer <a href="https://math.stackexchange.com/a/2452452/155881">here</a>.